In [9]:
## SETUP:
from trans_tools import *
from IrisBackendv3.codec.payload import CommandPayload
from IrisBackendv3.codec.packet import IrisCommonPacket
from IrisBackendv3.codec.metadata import DataPathway, DataSource
from IrisBackendv3.codec.magic import Magic
from IrisBackendv3.utils.basic import bytearray_to_spaced_hex as hexstr

In [3]:
import serial  # type: ignore # no type hints
import time
device: str = '/dev/ttyUSBx' # find correct name
baud: int = 115200

radio_buf = bytearray(b'')
# Hello System:
radio_buf.append(0x08) # MSG_TYPE: 0b0 (command/resp.), TECH_TYPE: 0b0001 (WiFi), Payload Size (Hi bits): 0b000
radio_buf.append(0x00) # Payload Size (Lo bits): 0x00
radio_buf.append(0x01) # CLASS_ID: 0x01 (SYSTEM)
radio_buf.append(0x02) # COMMAND_ID: 0x02 (HELLO)

boot_array = b'\x88\x0e\x01\x00\x01\x00\x03\x00\x00\x00\x55\x00\x05\x00\x00\x00\x05\x00'
print(f'Searching for boot array: {boot_array}')

try:
    serial_port = serial.Serial(device, baud)  # open serial port
    cprint("Connection Success!", 'green')
except serial.SerialException as e:
    cprint(
        f"Failed to connect to serial device. Is the device name right? Check the connection? Original error: {e}",
        'red')

boot_index: int = 0
found: bool = False
sent_cmd: bool = False
find_time = 0
ready: bool = serial_port is not None
try:
    while keep_running:= True and ready:
        b: Any = serial_port.read(1)

        if not found and b[0] == boot_array[boot_index]:
            boot_index += 1
            print(f"\t > Found more: {boot_index}")
        else:
            boot_index = 0

        if boot_index == len(boot_array):
            found = True
            find_time = time.time()
            print("\t > Found boot!")

        if found or boot_index > 0:
            print(f"{b}")

        if found and not sent_cmd:# and (time.time() - find_time) > 5:
            sent_cmd = True
            print(f"Writing HelloSystem() Command: {radio_buf}")
            time.sleep(3)
            
            if serial_port is not None:
                serial_port.write(bytes(radio_buf))
                print("\t > Written!")
            else:
                cprint(
                    "Can't send data, serial connection not started. Try `connect_serial()`.",
                    'red')

except KeyboardInterrupt:
        serial_port.close()

Searching for boot array: b'\x88\x0e\x01\x00\x01\x00\x03\x00\x00\x00U\x00\x05\x00\x00\x00\x05\x00'
Connection Success!
